In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def generarPuntosMuestra(numeroMuestrasClase1, numeroMuestrasClase2, nombreClase1, nombreClase2, listaCoeficientesCompletosHiperplano, minimoValorAbscisas, maximoValorAbscisas, numeroDimensiones = 2, margenMinimoSeparacionEntreClases = 1.0):
    muestraPuntos = []
    muestraPuntosPorClases = {nombreClase1: [], nombreClase2: []}
    
    #Se generan los puntos de la primera clase (no se tienen restricciones adicionales sobre las abscisas)
    for indice in range(numeroMuestrasClase1):
        abscisasPunto = []
        if numeroDimensiones > 1:
            abscisasPunto = list(np.random.uniform(low = minimoValorAbscisas, high = maximoValorAbscisas, size = numeroDimensiones - 1))
        puntoGenerado = abscisasPunto + [np.dot(listaCoeficientesCompletosHiperplano[0:(numeroDimensiones - 1)], abscisasPunto) + listaCoeficientesCompletosHiperplano[numeroDimensiones - 1] + margenMinimoSeparacionEntreClases + np.random.rand()]
        muestraPuntos.append(puntoGenerado)
        muestraPuntosPorClases[nombreClase1].append(puntoGenerado)
    
    #Se generan los puntos de la segunda clase (no se tienen restricciones adicionales sobre las abscisas)
    for indice in range(numeroMuestrasClase2):
        abscisasPunto = []
        if numeroDimensiones > 1:
            abscisasPunto = list(np.random.uniform(minimoValorAbscisas, maximoValorAbscisas, numeroDimensiones - 1))
        puntoGenerado = abscisasPunto + [np.dot(listaCoeficientesCompletosHiperplano[0:(numeroDimensiones - 1)], abscisasPunto) + listaCoeficientesCompletosHiperplano[numeroDimensiones - 1] - margenMinimoSeparacionEntreClases - np.random.rand()]
        muestraPuntos.append(puntoGenerado)
        muestraPuntosPorClases[nombreClase2].append(puntoGenerado)
    return muestraPuntos, muestraPuntosPorClases

def verificarMuestrasClasificadasDeUnaSolaClase(muestraPuntosPorClases, claseObjetivo, listaCoeficientesCompletosHiperplano, numeroDimensiones = 2):
#def revisarClasificacionLineal(clases, clase, w1, w2, b):
    numeroPuntosConSignoPositivo = 0
    numeroPuntosConSignoNegativo = 0
    for punto in muestraPuntosPorClases[claseObjetivo]:
        valorPunto = np.dot(listaCoeficientesCompletosHiperplano[0:(numeroDimensiones - 1)], punto[0:(numeroDimensiones - 1)]) + listaCoeficientesCompletosHiperplano[numeroDimensiones - 1]
        if valorPunto == 0.0:
            return False
        elif np.sign(valorPunto) == 1.0:
            numeroPuntosConSignoPositivo += 1
        else:
            numeroPuntosConSignoNegativo += 1
    
    if (numeroPuntosConSignoPositivo > 0 and numeroPuntosConSignoNegativo == 0) or (numeroPuntosConSignoPositivo == 0 and numeroPuntosConSignoNegativo > 0):
        return True
    return False
            
"""
clasificacion = []
for punto in clases[clase]:
    v = w1 * punto[0] + w2 * punto[1] + b
    if v != 0:
        clasificacion.append(np.sign(v))
    else:
        return 0
for resultado in clasificacion:
    if resultado != clasificacion[0]:
        return 0
return np.sign(clasificacion[0])

clasificacion = []
for punto in clases[clase]:
    v = w1 * punto[0] + w2 * punto[1] + b
    if v != 0:
        clasificacion.append(np.sign(v))
    else:
        return 0
for resultado in clasificacion:
    if resultado != clasificacion[0]:
        return 0
return np.sign(clasificacion[0])
"""

def separarPuntosPorListasDeCoordenadas(clases, clase):
    x = []
    y = []
    for punto in clases[clase]:
        x.append(punto[0])
        y.append(punto[1])
    return x, y

def obtenerListasCoordenadas(listaPuntos, numeroDimensiones = 2):
    diccionarioPuntosPorCoordenadas = {}
    for indice in range(numeroDimensiones):
        diccionarioPuntosPorCoordenadas[indice] = []
        
    for puntoActual in listaPuntos:
        for indice in range(numeroDimensiones):
            coordenadaActual = puntoActual[indice]
            diccionarioPuntosPorCoordenadas[indice].append(coordenadaActual)
    return diccionarioPuntosPorCoordenadas

#listaPuntosCamino = []
#listaValoresCoordenadasEjeX = []
#listaValoresCoordenadasEjeY = []
#for nodoActual in listaNodosCamino:
#    coordenadaXNodoActual, coordenadaYNodoActual = hashNumeroACoordenada(nodoActual, numeroVerticesAlto, numeroVerticesAncho)
#    listaValoresCoordenadasEjeX.append(coordenadaXNodoActual)
#    listaValoresCoordenadasEjeY.append(coordenadaYNodoActual)
#    listaPuntosCamino.append((coordenadaXNodoActual, coordenadaYNodoActual))
#return listaPuntosCamino, listaValoresCoordenadasEjeX, listaValoresCoordenadasEjeY

#Algoritmo de clasificacion para 2 clases linealmente separables
def algoritmoPerceptronSimple(muestra, clase1, clase2):
    w1 = np.random.rand()
    w2 = np.random.rand()
    b = np.random.rand()
    while w1 == 0 or w2 == 0 or b == 0:
        w1 = np.random.rand()
        w2 = np.random.rand()
        b = np.random.rand()
    alfa = 0.001
    plt.ion()
    plt.show()
    numeroPasos = 1
    while True:
        plt.clf()
        parejaSeleccionada = muestra[np.random.randint(len(muestra))]
        z = w1 * parejaSeleccionada[0] + w2 * parejaSeleccionada[1] + b
        if not ((parejaSeleccionada in clase1 and z >= 0) or (parejaSeleccionada in clase2 and z < 0)):
            if z < 0 and parejaSeleccionada in clase1:
                w1 += alfa * parejaSeleccionada[0]
                w2 += alfa * parejaSeleccionada[1]
                b += alfa
            else:
                if z >= 0 and parejaSeleccionada in clase2:
                    w1 -= alfa * parejaSeleccionada[0]
                    w2 -= alfa * parejaSeleccionada[1]
                    b -= alfa
        x1, y1 = separarPuntosPorListasDeCoordenadas(clases, '1')
        x2, y2 = separarPuntosPorListasDeCoordenadas(clases, '2')
        plt.plot(x1, y1, 'bo', label = 'Clase 1 de muestra')
        plt.plot(x2, y2, 'ro', label = 'Clase 2 de muestra')
        listaDeValoresEjeIndependiente = np.arange(minimoEjeX, maximoEjeX, 0.01)
        plt.plot(listaDeValoresEjeIndependiente, -(w1 * listaDeValoresEjeIndependiente + b) / w2, 'g-', label = "Frontera de Decision propuesta por el algoritmo")
        plt.legend(loc = 'upper left')
        plt.xlabel('Eje X')
        plt.ylabel('Eje Y')
        plt.title('Visualizacion de la recta que proporciona el algoritmo')
        plt.xlim(minimoEjeX, maximoEjeX)
        plt.ylim(minimoEjeY, maximoEjeY)
        plt.grid(True)
        plt.pause(tiempoPausa)
        #revisionClasificacion1 = revisarClasificacionLineal(clases, '1', w1, w2, b)
        #revisionClasificacion2 = revisarClasificacionLineal(clases, '2', w1, w2, b)
        revisionClasificacion1 = verificarMuestrasClasificadasDeUnaSolaClase(muestraPuntosPorClases, "Clase 1", listaCoeficientesPrincipalesHiperplano, interceptoHiperplano, numeroDimensiones = numeroDimensiones)
        revisionClasificacion2 = verificarMuestrasClasificadasDeUnaSolaClase(muestraPuntosPorClases, "Clase 2", listaCoeficientesPrincipalesHiperplano, interceptoHiperplano, numeroDimensiones = numeroDimensiones)
        numeroPasos += 1
        #if revisionClasificacion1 != 0 and revisionClasificacion2 != 0 and revisionClasificacion1 != revisionClasificacion2:
        if revisionClasificacion1 == True and revisionClasificacion2 == True:
            print("Fue(ron) necesario(s)", numeroPasos, "paso(s) para obtener una recta de clasificacion")
            print("La recta de clasificacion que proporciona el algoritmo es " + str(w1) + "x + " + str(w2) + "y + " + str(b) + " = 0")
            break
    plt.ioff()
    plt.show()

In [3]:
#tiempoPausa = 0.1
minimoEjeX = 0
maximoEjeX = 10
numeroDimensiones = 2
listaCoeficientesPrincipalesHiperplanoBase = [1.0] * (numeroDimensiones - 1)
coeficienteInterceptoHiperplanoBase = 0.0
nombreClase1 = "Clase 1"
nombreClase2 = "Clase 2"
margenMinimoSeparacionEntreClases = 0.5
numeroMuestrasClase1 = 10
numeroMuestrasClase2 = 10

In [4]:
listaCoeficientesCompletosHiperplanoBase = listaCoeficientesPrincipalesHiperplanoBase + [coeficienteInterceptoHiperplanoBase]
listaPuntosMuestra, muestraPuntosPorClases = generarPuntosMuestra(numeroMuestrasClase1, numeroMuestrasClase2, nombreClase1, nombreClase2, listaCoeficientesCompletosHiperplanoBase, minimoEjeX, maximoEjeX, numeroDimensiones = numeroDimensiones, margenMinimoSeparacionEntreClases = margenMinimoSeparacionEntreClases)
print(listaPuntosMuestra)

[[1.241184511447151, 2.023743267986415], [5.8041878730615295, 6.309910932915736], [2.558177389236622, 3.866358886217448], [5.920259041548361, 7.193216707366432], [0.38366837142592036, 1.0656039525620637], [5.2966934889194315, 5.921459423791039], [1.808534051221382, 2.4756756785130536], [0.7365916758742275, 1.8050337674596508], [6.657930524794339, 7.37521658132309], [0.3290876565430889, 1.6092296903348007], [7.614282873078104, 6.333438705791346], [5.978157264793037, 5.1983773870034895], [9.67764906277427, 8.623808024322178], [4.697558078839372, 3.3694673955680967], [7.328869333034507, 6.341258830627054], [0.8292743505924005, 0.2108382117052916], [0.9932517603358082, 0.11739680863925817], [3.859239057867022, 3.273592994352898], [8.051857125037913, 6.753976582452022], [5.93564706031926, 4.865330347299792]]


In [5]:
print(muestraPuntosPorClases)

{'Clase 1': [[1.241184511447151, 2.023743267986415], [5.8041878730615295, 6.309910932915736], [2.558177389236622, 3.866358886217448], [5.920259041548361, 7.193216707366432], [0.38366837142592036, 1.0656039525620637], [5.2966934889194315, 5.921459423791039], [1.808534051221382, 2.4756756785130536], [0.7365916758742275, 1.8050337674596508], [6.657930524794339, 7.37521658132309], [0.3290876565430889, 1.6092296903348007]], 'Clase 2': [[7.614282873078104, 6.333438705791346], [5.978157264793037, 5.1983773870034895], [9.67764906277427, 8.623808024322178], [4.697558078839372, 3.3694673955680967], [7.328869333034507, 6.341258830627054], [0.8292743505924005, 0.2108382117052916], [0.9932517603358082, 0.11739680863925817], [3.859239057867022, 3.273592994352898], [8.051857125037913, 6.753976582452022], [5.93564706031926, 4.865330347299792]]}


In [6]:
a = obtenerListasCoordenadas(listaPuntosMuestra, numeroDimensiones = numeroDimensiones)
print(a)

{0: [1.241184511447151, 5.8041878730615295, 2.558177389236622, 5.920259041548361, 0.38366837142592036, 5.2966934889194315, 1.808534051221382, 0.7365916758742275, 6.657930524794339, 0.3290876565430889, 7.614282873078104, 5.978157264793037, 9.67764906277427, 4.697558078839372, 7.328869333034507, 0.8292743505924005, 0.9932517603358082, 3.859239057867022, 8.051857125037913, 5.93564706031926], 1: [2.023743267986415, 6.309910932915736, 3.866358886217448, 7.193216707366432, 1.0656039525620637, 5.921459423791039, 2.4756756785130536, 1.8050337674596508, 7.37521658132309, 1.6092296903348007, 6.333438705791346, 5.1983773870034895, 8.623808024322178, 3.3694673955680967, 6.341258830627054, 0.2108382117052916, 0.11739680863925817, 3.273592994352898, 6.753976582452022, 4.865330347299792]}


In [7]:
algoritmoPerceptronSimple(listaPuntosMuestra, muestraPuntosPorClases[nombreClase2], muestraPuntosPorClases[nombreClase2])

NameError: name 'clases' is not defined

<Figure size 432x288 with 0 Axes>